In [1]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
from typing import Any, List, Dict
from time import time, sleep
import pprint
import copy
from dataclasses import dataclass

class GitHubClient:
    def __init__(self, token: str, base_url: str = "https://api.github.com", hourly_rate_limit: int = 5000):
        # Base attributes
        self.base_url: str = base_url
        self.token: str = token
        self.headers: dict = {
            "Authorization": f"token {self.token}"
        }
        # Rate limiting attributes
        self.last_search_time: float = 0.0
        self.hourly_rate_limit: int = hourly_rate_limit
        # Caching attributes
        self._cached_url: str|None = None
        self._cached_json_result: Any|None = None

    def _get(self, url: str, params: dict|None = None, raw: bool = False, cache: bool = False, **kwargs) -> Any:
        """
        Performs a get request to the provided url and returns the JSON response.
        The header does not need to be provided as it is handled by the class.
        If successful, the response is stored in the last_search attribute.
        It also respects the hourly rate limit set for the client. If the set hourly rate limit is reached, it waits before making the request.
        If raw is True, the raw response object is returned instead of the JSON response. In this case, caching is not performed and the value of cache is ignored.

        Args:
            url (str): the api endpoint to call
            params (dict, optional): query parameters to include in the request. Defaults to None.
            raw (bool, optional): whether to return the raw response object. Defaults to False. If True, caching is ignored.
            cache (bool, optional): whether to cache the response. Defaults to True.

        Returns:
            Dict: the JSON response from the API
        """
        # Check rate limit ("last time request made" + "waiting time between request" - "now")
        sleep(max(0, self.last_search_time + 3600 / self.hourly_rate_limit - time()))

        # Perform the GET request to the GitHub API and raise an error for bad responses
        response: requests.models.Response = requests.get(url, headers=self.headers, params=params)
        response.raise_for_status()

        # Parse the JSON response, store it and return it
        self.last_search_time = time()
        if raw:
            return response
        
        json_response: Dict = response.json()
        if cache:
            self._cached_url = response.url
            self._cached_json_result = copy.deepcopy(json_response)
        return json_response
    
    def _get_cached_response(self) -> Any|None:
        """
        Returns the cached JSON response from the last successful API call.

        Returns:
            Any|None: The cached JSON response or None if no request has been made yet
        """
        return copy.deepcopy(self._cached_json_result)
    
    def _get_last_page_number(self, response: requests.models.Response) -> int:
        """
        Extracts the last page number from the Link header of a paginated response.

        Args:
            response (requests.models.Response): The response object from the API call
        Returns:
            int: The last page number, or -1 if not found
        """
        if 'Link' in response.headers:
            for link in response.headers['Link'].split(','):
                if 'rel="last"' in link:
                    last_page_url = link.split(';')[0].strip('<> ')
                    return int(last_page_url.split('page=')[-1].split('&')[0])
        return -1
    
    def get_repository(self, owner: str, repo_name: str, commit_sha: str|None = None, params: dict|None = None, cache: bool = False) -> Dict:
        """
        Retrieves a specific repository from GitHub, eventually at a specific commit SHA.

        Args:
            owner (str): the owner of the repository
            repo_name (str): the name of the repository
            commit_sha (str, optional): the commit SHA to retrieve the repository at. Defaults to None.
            params (dict, optional): query parameters to include in the request. Defaults to None.
            cache (bool, optional): whether to cache the response. Defaults to False.
        Returns:
            Dict: The JSON response containing the repository information
        """
        url: str = f"{self.base_url}/repos/{owner}/{repo_name}"

        if commit_sha:
            url += f"/git/trees/{commit_sha}"

        return self._get(url, params=params, cache=cache)
    
    def get_repositories(self, query: str = "stars:>1000", sort: str|None = "stars", per_page: int = 10, page: int = -1, cache: bool = False) -> List[Dict]:
        """
        Searches for repositories on GitHub based on the provided query.
        If page is -1, it retrieves only all the results from all the pages. Hence, the results will not be cached.

        Args:
            query (str, optional): the search query. Defaults to "stars:>1000".
            sort (str|None, optional): the sort criteria. Defaults to "stars".
            per_page (int, optional): number of results per page. Defaults to 10.
            page (int, optional): the page number to retrieve. Defaults to -1.
            cache (bool, optional): whether to cache the response. Defaults to False.

        Returns:
            List[Dict]: The JSON response containing the search results
        """
        url: str = f"{self.base_url}/search/repositories"

        params: dict = {
            "q": query,
            "sort": sort,
            "order": "desc",
            "per_page": per_page,
            "page": page
        }
        if sort is None:
            params.pop("sort")

        if page == -1:
            all_items: List[Dict] = []

            params["per_page"] = 100
            params["page"] = 1

            raw_response = self._get(url, params=params, raw=True, cache=False)

            # Retrieve the number of pages with an api call
            last_page_number = min(self._get_last_page_number(raw_response), 10)

            all_items.extend(raw_response.json()["items"])

            # Retrieve all other pages since we already processed the first one
            for current_page in range(2, last_page_number + 1):
                repos = self.get_repositories(
                    query=query,
                    sort=sort,
                    per_page=100, # Max per_page value for GitHub API
                    page=current_page,
                    cache=False
                )

                all_items.extend(repos)

            return all_items

        return self._get(url, params=params, cache=cache)["items"]
    
    def get_releases(self, owner: str, repo_name: str, stable_only: bool = True, time_spaced: bool = True, number_of_releases: int = 8, cache: bool = False) -> List[Dict]:
        """
        Retrieves the releases of a specific repository.
        We can select to retrieve only stable releases (non pre-releases and non drafts).
        If time_spaced is True, the releases are spaced over time and only a certain number of releases are returned.

        Args:
            owner (str): the owner of the repository
            repo_name (str): the name of the repository
            stable_only (bool, optional): whether to retrieve only stable releases. Defaults to True.
            time_spaced (bool, optional): whether to space the releases over time. Defaults to True.
            number_of_releases (int, optional): number of releases to retrieve if time_spaced is True. Ignored if time_spaced is False. Defaults to 8.
            cache (bool, optional): whether to cache the response. Defaults to False.

        Returns:
            List[Dict]: The JSON response containing the releases
        """
        url: str = f"{self.base_url}/repos/{owner}/{repo_name}/releases"
        releases: List[Dict] = []

        for page in range(1, 1000):  # Arbitrary limit to avoid infinite loops
            params: dict = {
                "per_page": 100,
                "page": page
            }
            page_releases: List[Dict] = self._get(url, params=params, cache=cache)

            if not page_releases:  # No more releases available
                break

            releases.extend(page_releases)
        
        if stable_only:
            # Filter out pre-releases and drafts using filter()
            releases = list(filter(lambda r: not r.get("prerelease", False) and not r.get("draft", False), releases))

        # Sort releases by published date (or created date)
        releases.sort(key=lambda r: r["published_at"] or r["created_at"])

        if time_spaced and len(releases) > number_of_releases:
            # Convert dates to datetime objects for easier manipulation
            releases_with_dates = []
            for r in releases:
                date_str = r["published_at"] or r["created_at"]
                date_obj = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
                releases_with_dates.append((r, date_obj))

            # Determine the time span
            target_interval_days = (releases_with_dates[-1][1] - releases_with_dates[0][1]).days / (number_of_releases - 1)

            selected_releases_with_date = [releases_with_dates[0]]

            for i in range(1, number_of_releases - 1):
                target_date = selected_releases_with_date[0][1] + timedelta(days=int(i * target_interval_days))
                
                # Find the release closest to the target date
                closest_release = min(
                    releases_with_dates[1:],
                    key=lambda r: abs((r[1] - target_date).days)
                )
                selected_releases_with_date.append(closest_release)

            selected_releases_with_date.append(releases_with_dates[-1])

            # Ensure to keep only the release data, not the dates. Also ensure no more than the requested number of releases
            releases = [r[0] for r in selected_releases_with_date[:number_of_releases]]
            
        return releases

    def get_file_count_from_tree(self, owner: str, repo_name: str, default_branch: str) -> int:
        """
        Retrieves the number of files via the Git Tree API.
        If owner and repo_name are None, it uses the cached repository from the last API call.
        The call to the Git Tree API is never cached.

        Args:
            owner (str|None, optional): the owner of the repository. Defaults to None.
            repo_name (str|None, optional): the name of the repository. Defaults to None.
        Returns:
            int: The number of files in the repository
        """        
        # Retrieve the Git Tree recursively from the default branch
        tree_data: dict = self.get_repository(
            owner, repo_name, 
            commit_sha=default_branch, 
            params={"recursive": "1"}, 
            cache=False)
            
        # Count only files (not directories)
        file_count = len([item for item in tree_data["tree"] if item["type"] == "blob"])
        
        return file_count
    
    def get_release_count(self, owner: str, repo_name: str) -> int:
        """
        Retrieves the number of releases for a specific repository.

        Args:
            owner (str): the owner of the repository
            repo_name (str): the name of the repository
        Returns:
            int: The number of releases in the repository
        """
        url = f"{self.base_url}/repos/{owner}/{repo_name}/releases"

        response = self._get(url, params={"per_page": 1}, raw=True, cache=False)

        number_of_commits = self._get_last_page_number(response)
        if number_of_commits != -1: return number_of_commits
        
        return len(response.json())
    
    def get_contributor_count(self, owner: str, repo_name: str) -> int:
        """
        Retrieves the number of contributors for a specific repository.

        Args:
            owner (str): the owner of the repository
            repo_name (str): the name of the repository
        Returns:
            int: The number of contributors in the repository
        """
        url = f"{self.base_url}/repos/{owner}/{repo_name}/contributors"

        response = self._get(url, params={"per_page": 1, "anon": "true"}, raw=True, cache=False)

        number_of_commits = self._get_last_page_number(response)
        if number_of_commits != -1: return number_of_commits
        
        return len(response.json())

    def get_commit_count(self, owner: str, repo_name: str) -> int:
        """
        Retrieves the total number commits for a specific repository.

        Args:
            owner (str): the owner of the repository
            repo_name (str): the name of the repository
        Returns:
            int: The total number of commits in the repository
        """
        url = f"{self.base_url}/repos/{owner}/{repo_name}/commits"

        response = self._get(url, params={"per_page": 1}, raw=True, cache=False)

        number_of_commits = self._get_last_page_number(response)
        if number_of_commits != -1: return number_of_commits
        
        return len(response.json())


# Here the rate limit is set to 30000 requests per hour just for testing purposes
load_dotenv()
client = GitHubClient(token=os.getenv("GITHUB_TOKEN"), hourly_rate_limit=30000)

In [2]:
repos = client.get_repositories(query="machine learning", per_page=10, page=1, cache=False)
print(f"Nombre de projets récupérés : {len(repos)}")
pprint.pprint(repos)

Nombre de projets récupérés : 10
[{'allow_forking': True,
  'archive_url': 'https://api.github.com/repos/tensorflow/tensorflow/{archive_format}{/ref}',
  'archived': False,
  'assignees_url': 'https://api.github.com/repos/tensorflow/tensorflow/assignees{/user}',
  'blobs_url': 'https://api.github.com/repos/tensorflow/tensorflow/git/blobs{/sha}',
  'branches_url': 'https://api.github.com/repos/tensorflow/tensorflow/branches{/branch}',
  'clone_url': 'https://github.com/tensorflow/tensorflow.git',
  'collaborators_url': 'https://api.github.com/repos/tensorflow/tensorflow/collaborators{/collaborator}',
  'comments_url': 'https://api.github.com/repos/tensorflow/tensorflow/comments{/number}',
  'commits_url': 'https://api.github.com/repos/tensorflow/tensorflow/commits{/sha}',
  'compare_url': 'https://api.github.com/repos/tensorflow/tensorflow/compare/{base}...{head}',
  'contents_url': 'https://api.github.com/repos/tensorflow/tensorflow/contents/{+path}',
  'contributors_url': 'https://api

In [3]:
repos = client.get_repository(
    owner="tensorflow", 
    repo_name="tensorflow", 
    commit_sha="master", 
    params={"recursive": "1"},
    cache=False)
print("tree" in repos)
print(repos.keys())
#pprint.pprint(repos["tree"])

True
dict_keys(['sha', 'url', 'tree', 'truncated'])


In [4]:
release = client.get_releases("tensorflow", "tensorflow")
print(f"Nombre de releases récupérées pour tensorflow/tensorflow : {len(release)}")
pprint.pprint(release)
for r in release:
    print(f"- {r['tag_name']} published at {r['published_at']} (prerelease: {r['prerelease']}, draft: {r['draft']})")

Nombre de releases récupérées pour tensorflow/tensorflow : 8
[{'assets': [],
  'assets_url': 'https://api.github.com/repos/tensorflow/tensorflow/releases/2603983/assets',
  'author': {'avatar_url': 'https://avatars.githubusercontent.com/u/577277?v=4',
             'events_url': 'https://api.github.com/users/martinwicke/events{/privacy}',
             'followers_url': 'https://api.github.com/users/martinwicke/followers',
             'following_url': 'https://api.github.com/users/martinwicke/following{/other_user}',
             'gists_url': 'https://api.github.com/users/martinwicke/gists{/gist_id}',
             'gravatar_id': '',
             'html_url': 'https://github.com/martinwicke',
             'id': 577277,
             'login': 'martinwicke',
             'node_id': 'MDQ6VXNlcjU3NzI3Nw==',
             'organizations_url': 'https://api.github.com/users/martinwicke/orgs',
             'received_events_url': 'https://api.github.com/users/martinwicke/received_events',
           

In [5]:
from datetime import date

# Cette classe existe déjà dans le notebook 01b - Data storage.ipynb (c'est juste un copier-coller).
@dataclass
class RepositorySummary:
    name: str
    file_counts: int
    release_count: int
    size: int
    stars: int
    forks: int
    contributor_count: int
    created_at: date  # Nouvelle colonne
    updated_at: date  # Nouvelle colonne
    # Activité
    total_commits: int
    topics: list  # Changer de int à list pour stocker les topics

REPOSITORY_FEATURES = [
        "name", "file_counts", "release_count", "size", "stars", "forks", 
        "contributor_count", "created_at", "updated_at",
        "total_commits","topics"
    ]

from time import time, sleep


# Tests
repo = client.get_repository("tensorflow", "tensorflow")
print(client.get_file_count_from_tree(owner="tensorflow", repo_name="tensorflow", default_branch=repo["default_branch"])) # = 35431
print(client.get_release_count("tensorflow", "tensorflow")) # = 219
print(client.get_release_count("romeokienzler", "TensorFlow")) # = 0
print(client.get_contributor_count("tensorflow", "tensorflow")) # = 4985
print(client.get_contributor_count("romeokienzler", "TensorFlow")) # = 4
print(client.get_commit_count("tensorflow", "tensorflow")) # 189091




35446
219
0
4988
4
189174


In [6]:

from collections import defaultdict


SL = {
    ".js", ".html", ".css", ".java", ".py", ".cpp", ".rb", ".php", ".go", ".rs",
    ".vue", ".scss", ".cc", ".hpp", ".asm", ".s"
}

df_test = pd.DataFrame(columns=list(SL))
a = defaultdict(int, {".py": 1500, ".js": 3000, ".java": 500})

#self._df.loc[len(self._df)] = pd.Series(stats.__dict__)
df_test.loc[len(df_test)] = a
df_test.loc[len(df_test)] = {".vue": 1500, ".cc": 20, ".s": 500, ".html": 200, "": 300, ".tt": 1000}

df_test.fillna(0, inplace=True)
df_test

,.go,.cpp,.s,.html,.js,.hpp,.asm,.java,.rs,.scss,.php,.css,.py,.rb,.vue,.cc
0,0.0,0.0,0.0,0.0,3000.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,1500.0,0.0,0.0,0.0
1,0.0,0.0,500.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,20.0


In [7]:
from collections import defaultdict
from tqdm import tqdm, notebook
import pickle


SUPPORTED_LANGUAGES = {
    ".js", ".html", ".css", ".java", ".py", ".cpp", ".rb", ".php", ".go", ".rs",
    ".swift", ".kt", ".ts", ".cs", ".dart", ".lua", ".sh", ".pl", ".r", ".scala",
    ".hs", ".vb", ".m", ".jl", ".fs", ".ex", ".sql", ".h", ".c", ".jsx", ".tsx", 
    ".vue", ".scss", ".cc", ".hpp", ".asm", ".s"
}

class RepositoriesAnalytics:
    def __init__(self, client: GitHubClient, dataframe_path: str|None = None):
        self._client: GitHubClient = client

        if dataframe_path and os.path.exists(dataframe_path):
            self._repos_summary: pd.DataFrame = pd.read_csv(dataframe_path)
        else:
            self._repos_summary: pd.DataFrame = self._create_empty_repository_statistics_dataframe()

        self._language_matrices: Dict[str, pd.DataFrame] = {}

        self.existing_repositories: set = set(self._repos_summary["name"].tolist())

    def _create_empty_repository_statistics_dataframe(self) -> pd.DataFrame:
        """
        Creates an empty DataFrame with the appropriate columns for RepositoryStatistics.

        Returns:
            pd.DataFrame: An empty DataFrame with RepositoryStatistics columns
        """
        return pd.DataFrame(columns=REPOSITORY_FEATURES)
    
    def __getitem__(self, key: str) -> RepositorySummary:
        """
        Retrieves the RepositoryStatistics for a given repository name.

        Args:
            key (str): The name of the repository

        Returns:
            RepositoryStatistics: The statistics of the repository
        """
        key_stats = self._repos_summary[self._repos_summary["name"] == key]
        return RepositorySummary(**key_stats.iloc[0].to_dict())
    
    def __contains__(self, key: str) -> bool:
        """
        Checks if the repository with the given name exists in the internal DataFrame.

        Args:
            key (str): The name of the repository

        Returns:
            bool: True if the repository exists, False otherwise
        """
        return key in self.existing_repositories
    
    def __delitem__(self, key: str) -> None:
        """
        Deletes the repository with the given name and its language statistics from the internal DataFrames.

        Args:
            key (str): The name of the repository to delete
        """
        if key in self:
            self._repos_summary = self._repos_summary[self._repos_summary["name"] != key]
            del self._language_matrices[key]
            self.existing_repositories.remove(key)
    
    def to_pickle(self, path: str) -> None:
        """
        Saves the entire object to a pickle file at the specified path.

        Args:
            path (str): The file path to save the pickle
        """
        with open(path, "wb") as f:
            pickle.dump(self, f)

    def _append_repository_summary(self, client: GitHubClient, repo: dict) -> None:
        """
        Appends corresponding RepositoryStatistics to the internal DataFrame.

        Args:
            client (GitHubClient): The GitHub client to use for API calls
            repo (dict): The repository data from GitHub API
        """
        # Retrieve necessary information
        owner, repo_name = repo["owner"]["login"], repo["name"]

        # Get all statistics
        stats = RepositorySummary(
            name=repo["full_name"],
            file_counts=client.get_file_count_from_tree(owner=owner, repo_name=repo_name, default_branch=repo["default_branch"]),
            release_count=client.get_release_count(owner=owner, repo_name=repo_name),
            size=repo["size"],
            stars=repo["stargazers_count"],
            forks=repo["forks_count"],
            contributor_count=client.get_contributor_count(owner=owner, repo_name=repo_name),
            created_at=datetime.fromisoformat(repo.get("created_at", "")).date(),
            updated_at=datetime.fromisoformat(repo.get("updated_at", "")).date(),
            total_commits=client.get_commit_count(owner=owner, repo_name=repo_name),
            topics=repo.get("topics", [])
        )

        # Update the dataframe in place
        self._repos_summary.loc[len(self._repos_summary)] = pd.Series(stats.__dict__)

    def _add_language_matrix(self, client: GitHubClient, repo: dict) -> bool:
        """
        Appends the language statistics DataFrame to the internal dictionary for a given repository.

        Args:
            client (GitHubClient): The GitHub client to use for API calls
            repo (dict): The repository data from GitHub API
        Returns:
            bool: True if at least one release was processed, False otherwise
        """
        # Retrieve necessary information
        owner, repo_name, full_name = repo["owner"]["login"], repo["name"], repo["full_name"]

        # Get 8 time-spaced stable releases
        releases = client.get_releases(owner, repo_name, stable_only=True, time_spaced=True, number_of_releases=8)

        if not releases:
            return False

        # Process each release to compute the percentage of each language
        for release in releases:
            # Retrieve the Git Tree recursively from the release tag
            tree_data = client.get_repository(
                owner, 
                repo_name, 
                commit_sha=release['tag_name'], 
                params={"recursive": "1"}, 
                cache=False
            )['tree']

            # Compute file sizes per language
            file_sizes = defaultdict(int)
            for file_info in filter(lambda f: f['type'] == 'blob', tree_data):
                file_extension = os.path.splitext(file_info['path'])[1]
                file_size = file_info['size']

                if file_extension not in SUPPORTED_LANGUAGES:
                    continue

                file_sizes[file_extension] += file_size

            # Normalize file sizes
            total_size = sum(file_sizes.values())
            for ext in file_sizes: 
                file_sizes[ext] /= total_size
                file_sizes[ext] = round(file_sizes[ext], 4)

            # Create a temporary dict to include the date
            temp_dict = dict(file_sizes)
            temp_dict['date'] = datetime.fromisoformat(release.get('published_at', release.get('created_at', ''))).date()

            # Create the dataframe if not existing
            if full_name not in self._language_matrices:
                self._language_matrices[full_name] = pd.DataFrame(columns=['date']+list(SUPPORTED_LANGUAGES))

            # Effectively add the row to the DataFrame
            num_rows = len(self._language_matrices[full_name])
            self._language_matrices[full_name].loc[num_rows] = temp_dict

            # Replace NaN with 0
            self._language_matrices[full_name].fillna(0, inplace=True)

        return True

    def collect_repository_data_for_search(self, client: GitHubClient, query: str, sort: str, update: bool = False):
        """
        Collects repository data for repositories matching the search query.

        Args:
            client (GitHubClient): The GitHub client to use for API calls
            query (str): The search query to find repositories
            sort (str): The sort criteria for the search results
            update (bool, optional): Whether to update existing repositories. Defaults to False.
        """

        repositories = client.get_repositories(query=query, sort=sort, per_page=10, page=1)
        #repositories = client.get_repositories(query=query, sort=sort, page=-1)

        processed_repo = 0

        for repo in notebook.tqdm(repositories):
            # Skip if already exists and not updating
            if repo["full_name"] in self and not update:
                continue
            # If updating, remove existing entry
            if update:
                del self[repo["full_name"]]

            # Compute and store language matrix
            any_release = self._add_language_matrix(client, repo)
                
            # Skip if no release found
            if not any_release:
                continue
            
            # Get and append repository summary to the internal DataFrame
            self._append_repository_summary(client, repo)

            # Update the set of existing repositories
            self.existing_repositories.add(repo["full_name"])

            processed_repo += 1

        # Return the actual number of processed repositories
        return processed_repo


    def repositories_summary(self) -> pd.DataFrame:
        """
        Returns the internal DataFrame containing the repository statistics.

        Returns:
            pd.DataFrame: DataFrame containing the repository statistics
        """
        return self._repos_summary

analytics = RepositoriesAnalytics(client=client)
start = time()
processed_count = analytics.collect_repository_data_for_search(client, query="machine learning", sort="stars")
end = time()
print(f"Temps écoulé pour collecter {processed_count} repositories : {end - start} secondes")
df_summary = analytics.repositories_summary()

  0%|          | 0/10 [00:00<?, ?it/s]

Temps écoulé pour collecter 5 repositories : 85.24972200393677 secondes


In [8]:
df_summary

,name,file_counts,release_count,size,stars,forks,contributor_count,created_at,updated_at,total_commits,topics
0,tensorflow/tensorflow,35446,219,1246899,193292,75157,4988,2015-11-07,2026-01-11,189174,"[deep-learning, deep-neural-networks, distribu..."
1,huggingface/transformers,5254,236,437636,154918,31693,3722,2018-10-29,2026-01-11,21650,"[audio, deep-learning, deepseek, gemma, glm, h..."
2,scikit-learn/scikit-learn,1736,49,179130,64590,26589,3456,2010-08-17,2026-01-11,33295,"[data-analysis, data-science, machine-learning..."
3,gradio-app/gradio,3231,4402,314745,41267,3224,461,2018-12-19,2026-01-11,8224,"[data-analysis, data-science, data-visualizati..."
4,lutzroeder/netron,252,3,79995,32161,3058,1,2010-12-26,2026-01-11,9641,"[ai, coreml, deep-learning, deeplearning, kera..."


In [9]:
analytics._language_matrices["tensorflow/tensorflow"]

,date,.pl,.cpp,.html,.sql,.hpp,.kt,.lua,.asm,.java,...,.php,.css,.hs,.py,.r,.rb,.swift,.ts,.fs,.jsx
0,2016-02-16,0.0000,0.0000,0.0444,0.0,0.0,0.0,0.0,0.0,0.0035,...,0.0,0.0,0.0,0.1982,0.0,0.0,0.0000,0.0187,0.0,0.0
1,2017-06-30,0.0000,0.0000,0.0453,0.0,0.0,0.0,0.0,0.0,0.0066,...,0.0,0.0,0.0,0.3985,0.0,0.0,0.0000,0.0317,0.0,0.0
2,2018-11-05,0.0001,0.0000,0.0487,0.0,0.0,0.0,0.0,0.0,0.0096,...,0.0,0.0,0.0,0.4036,0.0,0.0,0.0000,0.0000,0.0,0.0
3,2020-01-26,0.0001,0.0202,0.0360,0.0,0.0,0.0,0.0,0.0,0.0068,...,0.0,0.0,0.0,0.3606,0.0,0.0,0.0004,0.0000,0.0,0.0
4,2021-08-10,0.0001,0.0000,0.0312,0.0,0.0,0.0,0.0,0.0,0.0065,...,0.0,0.0,0.0,0.2895,0.0,0.0,0.0004,0.0000,0.0,0.0
5,2022-11-18,0.0000,0.0000,0.0277,0.0,0.0,0.0,0.0,0.0,0.0065,...,0.0,0.0,0.0,0.2512,0.0,0.0,0.0005,0.0000,0.0,0.0
6,2024-03-08,0.0000,0.0000,0.0246,0.0,0.0,0.0,0.0,0.0,0.0062,...,0.0,0.0,0.0,0.2385,0.0,0.0,0.0004,0.0000,0.0,0.0
7,2025-08-13,0.0000,0.0009,0.0345,0.0,0.0,0.0,0.0,0.0,0.0054,...,0.0,0.0,0.0,0.2076,0.0,0.0,0.0004,0.0000,0.0,0.0


In [10]:
analytics._language_matrices["scikit-learn/scikit-learn"]

,date,.pl,.cpp,.html,.sql,.hpp,.kt,.lua,.asm,.java,...,.php,.css,.hs,.py,.r,.rb,.swift,.ts,.fs,.jsx
0,2016-04-17,0.0,0.0685,0.0004,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0104,0.0,0.1996,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-10-22,0.0,0.0158,0.0015,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0357,0.0,0.8708,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-12-20,0.0,0.0144,0.0014,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0320,0.0,0.8828,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-05-19,0.0,0.0134,0.0042,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0460,0.0,0.9030,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-10-25,0.0,0.0123,0.0028,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0154,0.0,0.9550,0.0,0.0,0.0,0.0,0.0,0.0
5,2023-03-09,0.0,0.0111,0.0026,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0139,0.0,0.9562,0.0,0.0,0.0,0.0,0.0,0.0
6,2024-07-03,0.0,0.0103,0.0013,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0008,0.0,0.9780,0.0,0.0,0.0,0.0,0.0,0.0
7,2025-12-10,0.0,0.0094,0.0012,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0009,0.0,0.9789,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def get_language_usage_for_commit():
    pass

In [12]:
def create_language_matrix_for_repository():
    pass